In [1]:
import pandas as pd
import numpy as np
import os
import GPy
import contextlib
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

In [2]:
folder_path = '/Users/pmzff/Documents/GitHub/ModularCircFF/Emulation'

In [3]:
# Read Input Data
df_x = pd.read_csv(f'{folder_path}/Input/input_100.csv')

# Select varying inputs only
#columns_with_multiple_values = df_x.nunique() > 1
#filtered_input = df_x.loc[:, columns_with_multiple_values]

# Select only first 5 inputs 
filtered_input = df_x.iloc[:,:5]

dataframes = {}

# Read PCA data
for i in range(3):
    df_name = f'y_PC{i+1}'  # Create the dataframe name
    dataframes[df_name] = pd.read_csv(f'{folder_path}/Outputs/Out_fixed/PCA/PC{i+1}.csv')  # Read and store the dataframe
    dataframes[f'all_{df_name}'] =  pd.read_csv(f'{folder_path}/Outputs/Out_fixed/PCA/all_PC{i+1}.csv') 
    dataframes[f'CO_{df_name}'] =  pd.read_csv(f'{folder_path}/Outputs/Out_fixed/PCA/CO_PC{i+1}.csv') 

y_PC1 = dataframes['y_PC1']
y_PC2 = dataframes['y_PC2']
y_PC3 = dataframes['y_PC3']

all_y_PC1 = dataframes['all_y_PC1']
all_y_PC2 = dataframes['all_y_PC2']
all_y_PC3 = dataframes['all_y_PC3']

CO_y_PC1 = dataframes['CO_y_PC1']
CO_y_PC2 = dataframes['CO_y_PC2']
CO_y_PC3 = dataframes['CO_y_PC3']

df_pressure = pd.read_csv(f'{folder_path}/Outputs/Out_fixed/pressure_traces_r_pat/all_pressure_traces.csv')
dataframes['CO'] = df_pressure.iloc[:,100:101]

dataframes['mean_press']= df_pressure.iloc[:,:100].mean(axis=1).to_frame(name='mean_press')
dataframes['max_press']= df_pressure.iloc[:,:100].max(axis=1).to_frame(name='max_press')
dataframes['min_press']= df_pressure.iloc[:,:100].min(axis=1).to_frame(name='min_press')


list(dataframes.keys())

['y_PC1',
 'all_y_PC1',
 'CO_y_PC1',
 'y_PC2',
 'all_y_PC2',
 'CO_y_PC2',
 'y_PC3',
 'all_y_PC3',
 'CO_y_PC3',
 'CO',
 'mean_press',
 'max_press',
 'min_press']

In [4]:
filtered_input

,# svn.c,pat.r,pat.c,rv.E_act,T
0,27.687002,0.443723,3.278327,1.623082,0.478369
1,10.935100,0.243385,4.286711,2.717826,0.805784
2,18.671167,0.356841,2.549143,0.822205,0.950488
3,24.992088,0.155291,5.101243,3.333386,0.614289
4,22.700741,0.326891,3.862239,2.779415,0.551895
...,...,...,...,...,...
95,27.406567,0.196824,3.333852,1.704984,0.646159
96,26.960391,0.395660,2.043640,0.645535,0.810764
97,10.379113,0.271468,5.547834,3.156299,0.473453
98,17.956148,0.327547,3.782205,1.440794,0.618088


In [5]:
# Read in dataframe of fitted linear models
model_dataframes = pd.read_pickle(f'{folder_path}/Outputs/Emulators/linear_models_and_r2_scores.csv')

model_dataframes

,R2_Score,Model
y_PC1,0.987922,LinearRegression()
all_y_PC1,0.987971,LinearRegression()
CO_y_PC1,0.987922,LinearRegression()
y_PC2,0.882277,LinearRegression()
all_y_PC2,0.888562,LinearRegression()
CO_y_PC2,0.882277,LinearRegression()
y_PC3,0.647736,LinearRegression()
all_y_PC3,0.873160,LinearRegression()
CO_y_PC3,0.647736,LinearRegression()
CO,0.940900,LinearRegression()


In [6]:
# Read input and output data
X = filtered_input.copy()

# Initialize a dictionary to store results
inverse_results_min_norm = {}
inverse_results_y = {}

# Loop through each row in the model_dataframes
for index, row in model_dataframes.iterrows():
    model = row['Model']  # Assuming the column containing the fitted models is named 'model'
    
    Y = dataframes[index].copy()
  
      
    # Split the data into training and testing sets (80% train, 20% test)
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

    coeffs = model.coef_[0][None].T
    intercept = model.intercept_
    
    # Scale y_obs by intercept
    scale_y_obs = y_test - intercept
    
    # Compute the pseudo-inverse of the coefficient matrix
    beta_inv = np.linalg.inv(coeffs.T @ coeffs) @ coeffs.T
    x_hat = scale_y_obs @ beta_inv
    
    x_hat_headers = [f'{col}_\u0302' for col in X_test.columns]
    x_hat.columns = x_hat_headers
    
    # Combine X_test and x_hat into a dataframe
    df = pd.concat([X_test, x_hat], axis=1)
    inverse_results_min_norm[f'model_{index}'] = df

    y_calibrated = (x_hat @ coeffs) + intercept 
    y_compare = pd.concat([y_test, y_calibrated], axis=1)
    y_compare.columns = ("y_true", "y_calibrated")
    inverse_results_y[f'model_{index}'] = y_compare

In [7]:
inverse_results_y


{'model_y_PC1':        y_true  y_calibrated
 83 -13.659440    -13.659440
 53 -15.954471    -15.954471
 70   8.896375      8.896375
 45  -8.088185     -8.088185
 44  -2.822037     -2.822037
 39 -11.478398    -11.478398
 22   4.176664      4.176664
 80   8.225622      8.225622
 10  14.414039     14.414039
 0   14.672208     14.672208
 18  11.567602     11.567602
 30  20.592315     20.592315
 73   1.136444      1.136444
 33 -13.713178    -13.713178
 90  13.088492     13.088492
 4    4.124239      4.124239
 76   3.779777      3.779777
 77 -14.093330    -14.093330
 12  11.222030     11.222030
 31 -11.062753    -11.062753,
 'model_all_y_PC1':        y_true  y_calibrated
 83 -13.670844    -13.670844
 53 -15.958746    -15.958746
 70   8.908210      8.908210
 45  -8.102175     -8.102175
 44  -2.824968     -2.824968
 39 -11.450656    -11.450656
 22   4.167202      4.167202
 80   8.227045      8.227045
 10  14.419235     14.419235
 0   14.700361     14.700361
 18  11.553899     11.553899
 30  20.

In [8]:
for index in inverse_results_y:
    print(index)
    mse = np.mean((inverse_results_y[index].loc[:, 'y_calibrated'] - inverse_results_y[index].loc[:, 'y_true'])**2)
    # print(f'{mse:.5f}')
    print(mse)

model_y_PC1
6.55740627464966e-30
model_all_y_PC1
3.540259831212172e-29
model_CO_y_PC1
6.55740627464966e-30
model_y_PC2
1.1174245537334743e-31
model_all_y_PC2
5.669937756276022e-32
model_CO_y_PC2
1.1174245537334743e-31
model_y_PC3
8.01765237701514e-31
model_all_y_PC3
1.3698754508437532e-30
model_CO_y_PC3
8.01765237701514e-31
model_CO
1.873544649899903e-31
model_mean_press
1.8932661725304283e-30
model_max_press
3.31321580192825e-30
model_min_press
4.575393250281868e-30
